In [69]:
from dotenv import load_dotenv
import os
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor


import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from sendgrid import SendGridAPIClient

In [63]:
load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
sendgrid_api_key = os.getenv("SENDGRID_API_KEY")
if not groq_api_key:
    print("Error: GROQ_API_KEY not found in environment variables.")
    exit(1)
if not sendgrid_api_key:
    print("Error: SENDGRID_API_KEY not found in environment variables.")
    exit(1)

In [64]:
llm = ChatOpenAI(
    model="llama3-70b-8192",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

In [65]:
instructions = {
    "agent1": """You are a sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write professional, serious cold emails. Do not give anything else in the response except the Email.""",
    "agent2": """You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write witty, engaging cold emails that are likely to get a response. Do not give anything else in the response except the Email.""",
    "agent3": """You are a busy sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write concise, to-the-point cold emails. Do not give anything else in the response except the Email.""",
    "picker": """You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to. Do not give an explanation; reply with the selected email only.""",
    "sales_manager": """You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools (agent1_tool, agent2_tool, agent3_tool) to generate three different email drafts. Do not proceed until all three drafts are ready.
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
3. Use the send_email tool to send the best email (and only the best email) to the user.
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one."""
}


In [66]:
memory_agent1 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
@tool
def agent1_tool(input: str) -> str:
    """Professional cold email agent for ComplAI. Returns a serious cold email based on the user's input."""
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent1"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    return chain.invoke({"input": input, "chat_history": memory_agent1.buffer}).content


In [67]:
memory_agent2 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
@tool
def agent2_tool(input: str) -> str:
    """Humorous cold email agent for ComplAI. Returns a witty and engaging cold email based on the user's input."""
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent2"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    return chain.invoke({"input": input, "chat_history": memory_agent2.buffer}).content

In [68]:
memory_agent3 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
@tool
def agent3_tool(input: str) -> str:
    """Concise cold email agent for ComplAI. Returns a short, to-the-point cold email based on the user's input."""
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent3"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    return chain.invoke({"input": input, "chat_history": memory_agent3.buffer}).content

In [ ]:
# def run_all_agents(input_prompt: str) -> list:
#     tools = [agent1_tool, agent2_tool, agent3_tool]
#     results = []
    
#     for tool in tools:
#         try:
#             result = tool.invoke(input_prompt)
#             results.append(result)
#         except Exception as e:
#             print(f"Error running {tool.name}: {str(e)}")
#             results.append(f"Error: {str(e)}")
    
#     return results

In [ ]:
# prompt = "Write a cold sales email"
# labels = ["Professional Email", "Humorous Email", "Concise Email"]
# drafts = run_all_agents(prompt)

# for i, (label, draft) in enumerate(zip(labels, drafts), start=1):
#     print(f"\n=== {label} (Draft {i}) ===\n")
#     print(draft)
#     print("\n" + "=" * 40 + "\n")


=== Professional Email (Draft 1) ===

Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Prospect's Name],

I came across your company, [Company Name], and was impressed with the innovative solutions you're bringing to the [Industry/Market] space. As a trusted partner to numerous organizations in this sector, I'm aware of the importance of maintaining robust security controls and complying with industry standards, such as SOC 2.

As you prepare for your upcoming audit, I understand the challenges that come with manually managing compliance. The time-consuming task of gathering evidence, identifying gaps, and implementing remediation measures can be overwhelming, taking away from your core business focus.

That's where ComplAI comes in. Our cutting-edge SaaS tool leverages AI to simplify and automate SOC 2 compliance, ensuring you're always audit-ready. With ComplAI, you can:

* Automate evidence collection and mapping to relevant controls
* Identify and prioritize gaps in y

In [ ]:
# sengrid_api_key = os.getenv("SENDGRID_API_KEY")

In [ ]:
# def send_test_email():
#     sg = sendgrid.SendGridAPIClient(api_key=sengrid_api_key)
#     from_email = Email("dabhideep44@gmail.com")  
#     to_email = To("dabhideep424@gmail.com")  
#     content = Content("text/plain", "This is an important test email")
#     mail = Mail(from_email, to_email, "Test email", content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     print(response.status_code)

# send_test_email()

202


In [ ]:
# memory_picker = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# @tool
# def sales_picker(input: str) -> str:
#     """Best email picker"""
#     prompt = ChatPromptTemplate.from_messages([
#         SystemMessagePromptTemplate.from_template(instructions["picker"]),
#         HumanMessagePromptTemplate.from_template("{input}")
#     ])
#     chain = RunnableSequence(prompt | llm)
#     return chain.invoke({"input": input, "chat_history": memory_picker.buffer}).content

In [ ]:
# input_for_picker = "\n\n---\n\n".join(drafts)  
# selected_email = sales_picker.invoke(input_for_picker)
# print("\n=== Selected Email ===\n")
# print(selected_email)
# print("\n" + "=" * 40 + "\n")


=== Selected Email ===

Subject: Are Auditors Your Arch-Nemesis Too?

Dear [First Name],

If the thought of an impending SOC2 audit sends you into a tailspin of dread, rest assured you're not alone! We've all been there – pouring over spreadsheets, praying to the compliance gods, and mainlining coffee like it's oxygen.

But what if I told you there's a way to tame the audit beast, and actually enjoy the process (okay, maybe "enjoy" is a stretch, but at least not lose sleep over it)? 

Our AI-powered SOC2 compliance tool, ComplAI, is here to save the day (and your sanity)! With automated workflows, real-time monitoring, and actionable insights, we'll have you ready for that audit in no time.

Curious how it works? I'd love to set up a quick call to discuss the magic of ComplAI.

Hit reply, and let's get this compliance party started!

Cheers,

[Your Name]

P.S. If you're still using spreadsheets, don't worry, your secret is safe with me




In [ ]:
@tool
def send_email(email_content: str) -> str:
    """sends the selected email to the reciepent's email using SendGrid"""
    try:
        message = Mail(
            from_email = Email("dabhideep44@gmail.com"),
            to_email = To("dabhideep424@gmail.com"),
            subject = "Cold Sales Email from ComplAI",
            plain_text_content = Content("text/plain", email_content)
        )
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)
        return f"Email sent successfully: {email_content[:50]}..."
        
    except Exception as e:
        return f"Error sending email: {str(e)}"

In [75]:
memory_sales_manager = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
def sales_manager(input_prompt: str) -> str:
    """Sales Manager agent that generates, selects, and sends the best email."""
    tools = [agent1_tool, agent2_tool, agent3_tool]
    drafts = []
    for tool in tools:
        try:
            result = tool.invoke(input_prompt)
            drafts.append(result)
        except Exception as e:
            drafts.append(f"Error: {str(e)}")

    drafts_input = "\n\n---\n\n".join(drafts)

    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["sales_manager"]),
        HumanMessagePromptTemplate.from_template("Input prompt: {input_prompt}\nDrafts:\n{drafts}")
    ])
    chain = RunnableSequence(prompt | llm.bind_tools([agent1_tool, agent2_tool, agent3_tool, send_email]))
    result = chain.invoke({"input_prompt": input_prompt, "drafts": drafts_input})

    return result.content

In [76]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "SalesManagerProject"

In [77]:
message = "Send a cold sales email addressed to 'Dear CEO'"
result = sales_manager(message)
print("\n=== Sales Manager Result ===\n")
print(result)
print("\n" + "=" * 40 + "\n")


=== Sales Manager Result ===




